In [ ]:
"""Please keep in mind that this whole document is only
to give a general idea of what to implement. 
These are not functioning codes."""

In [ ]:
%pip install "requests==2.28.1"

In [1]:
import sys
from urllib.parse import urljoin
import requests
sys.path.insert(1, '/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/slang-translator/Connection')
    
from elastic_con import client 
import nltk

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>


In [63]:
import nltk

In [73]:
import spacy
spacy.load("en_core_web_sm")

In [3]:
from gensim.parsing.preprocessing import remove_stopwords

In [65]:
from spacy.matcher import Matcher
from spacy.util import filter_spans

In [5]:
from elasticsearch import Elasticsearch

In [74]:
nlp = spacy.load("en_core_web_sm")

In [7]:
# draft : implementation step
######################################
# translate regular sentence to slang#
######################################

input_str = "Thanks for taking the time to post. It appears the making of a nation is missing."

In [8]:
str_doc = nlp(input_str)

In [69]:
len(str_doc)

18

In [70]:
print(str(list(str_doc.noun_chunks)))

[Thanks, the time, It, the making, a nation]


In [77]:
def noun_phrases_extract(nlp_result):
    # NOTE: the results of spacy for eg
    # .noun_chunks is constituted of elements 
    # rather than simple strings 
    noun_lst = nlp_result.noun_chunks
    return list(noun_lst)

In [78]:
def verb_phrases_extract(nlp_result):
    # under a different scheme
    # acknowledgement: 
    # https://stackoverflow.com/questions/47856247/extract-verb-phrases-using-spacy
    pattern = [[{'POS': 'VERB', 'OP': '?'},{'POS': 'ADV', 'OP': '*'}, # additional wildcard - match any text in between
               {'POS': 'VERB', 'OP': '+'}]]
    print(type(pattern))
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb phrase", pattern)
    matches = matcher(nlp_result)
    spans = [nlp_result[start:end] for _, start, end in matches]
    return filter_spans(spans)
    # preserve the parsed positions of words so that the -ing forms
    # won't get messed up

In [79]:
noun_phrases_extract(nlp("I am really in need of a break."))

[I, need, a break]

In [ ]:
""" kopfschmerzen: how to translate "I am really in need of a break" to 
" I rnab" for eg if rnab=really need a break is some kind of legal slang. 
IDK if fuzzy search can do that and I hope urban dic has abbreviations 2"""

In [62]:
# NOTE: not finished.
def phrase_extraction(nlp_result):
    # lemmatize first, extract phrases later
    lst_sentence = []
    for token in nlp_result:
        print(token.pos_)
        word = token.lemma_
        lst_sentence.append(word)
    sentence = " ".join(lst_sentence)
    nlp_result = nlp(sentence)
    print(nlp_result)
    noun_list = noun_phrases_extract(nlp_result)
    verb_list = verb_phrases_extract(nlp_result)
    lst_phrase_temp = noun_list+verb_list
    return lst_phrase_temp

In [80]:

st1 = 'i will laugh out really loud or i told her never ever to give up'
s1 = nlp(st1)

In [81]:
phrase_extraction(s1)
# doing an adequate job but can be better

PRON
AUX
VERB
ADP
ADV
ADV
CCONJ
PRON
VERB
PRON
ADV
ADV
PART
VERB
ADP
I will laugh out really loud or I tell she never ever to give up
<class 'list'>


[I, I, she, laugh, tell, give]

In [66]:
"I love".split(" ")

['I', 'love']

In [64]:
verb_phrases_extract(str_doc)[0]
# NOTE: theres room for improvement in accuracy

<class 'list'>


'taking'

In [ ]:
"""To find and return subject of a preprocessed sentence."""
# I wrote the function based on which one may
# omit the subject of a sentence with a possibility. 
# This is not used now, as I still don't
# know how to calculate the weight of such omission.
def subject_find(doc):
    for token in doc:
        if "subj" in token.dep_:
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [ ]:
# do slang dictionary query with part of the input sentence
"""returns all possible k-word-shingle for the input sentence.
form: list, [[1-shingle][2-shingle]...[k-shingle]]""" 
def multiple_shingle(doc_str):
    doc = doc_str.split(" ")
    shingle_lst=[]
    for k in range(0,len(doc)):
        shingle_lst_temp = []
        for i in range(len(doc)-k):
            shingle_lst_temp.append(doc[i:k+i+1])
        shingle_lst.append(shingle_lst_temp)
    return shingle_lst

In [ ]:
multiple_shingle(input_str)

[[['I'], ['have'], ['something'], ['to'], ['say']],
 [['I', 'have'], ['have', 'something'], ['something', 'to'], ['to', 'say']],
 [['I', 'have', 'something'],
  ['have', 'something', 'to'],
  ['something', 'to', 'say']],
 [['I', 'have', 'something', 'to'], ['have', 'something', 'to', 'say']],
 [['I', 'have', 'something', 'to', 'say']]]

In [ ]:
# TODO: perform lemmatization to all the shingles with POS tag

In [8]:
index_name = "slang-demo-json"
request = '''{"query": {"match_all": {}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'slang-demo-json', '_type': '_doc', '_id': '0', '_score': 1.0, '_source': {'id': 0, 'acronym': '2day', 'meaning': 'today'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '1', '_score': 1.0, '_source': {'id': 1, 'acronym': '2m2h', 'meaning': 'too much too handle'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '2', '_score': 1.0, '_source': {'id': 2, 'acronym': '2moro', 'meaning': 'tomorrow'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '3', '_score': 1.0, '_source': {'id': 3, 'acronym': '2nite', 'meaning': 'tonight'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '4', '_score': 1.0, '_source': {'id': 4, 'acronym': '4eae', 'meaning': 'for ever and ever'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '5', '_score': 1.0, '_source': {'id': 5, 'acronym': 'aaf', 'meaning': 'always and forever'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '6', '_score': 1.0, '_source': {'id': 6, 'acronym': 'aar', 'meaning': 'at any rate'}

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
request = '''{"query": {"match": { "meaning":"i really need a break!"}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'slang-demo-json', '_type': '_doc', '_id': '1181', '_score': 11.119665, '_source': {'id': 1181, 'acronym': 'inah', 'meaning': 'i need a hug'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '769', '_score': 9.979803, '_source': {'id': 769, 'acronym': 'gmab', 'meaning': 'give me a break'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '1785', '_score': 9.746098, '_source': {'id': 1785, 'acronym': 'nal', 'meaning': 'need a life'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '1192', '_score': 8.772346, '_source': {'id': 1192, 'acronym': 'iny', 'meaning': 'i need you'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '1170', '_score': 8.5358715, '_source': {'id': 1170, 'acronym': 'imr', 'meaning': 'i mean really'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '1188', '_score': 7.854871, '_source': {'id': 1188, 'acronym': 'intk', 'meaning': 'i need to know'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '1875', '_score': 7.8

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  


In [17]:
s1 = 'i really need a break!'

In [59]:
testvar = '''{"query": {"match": { "meaning": }}}'''
testvar = """ '''{\"query\": {\"match\": { \"meaning\": """ + s1 + """ }}}'''"""
testvar

#testvar = '''{"query": {"match": { "meaning": }}}'''

# request = '''{"query": {"match": { "meaning": }}}'''
#results = client.search(index=index_name, body=testvar,size = 100)['hits']['hits']

' \'\'\'{"query": {"match": { "meaning": i really need a break! }}}\'\'\''

In [16]:
for d in results:
    print(d['_source']['acronym'],'-->',d['_source']['meaning'])

inah --> i need a hug
gmab --> give me a break
nal --> need a life
iny --> i need you
imr --> i mean really
intk --> i need to know
nism --> need i say more
irly --> i really like you
hbk --> heart break kid
omb --> on my break
idrc --> i do not really care
idrk --> i do not really know
irdc --> i really do not care
irdk --> i really do not know
orly --> oh really
nmi --> need more info
ntk --> need to know
bnr --> but not really
lrl --> laughing really loud
lyr --> love you really
psr --> pretty standard really
rog --> really old git
ihac --> i have a customer
nnto --> no need to open
nntr --> no need to reply
wntt --> we need to talk
lorl --> laugh out really loud
rdc --> really do not care
ianal --> i am not a lawyer 
ianad --> i am not a doctor
ioab --> i am on a boat
dntk --> do not need to know
alihal --> at least i have a life
idgad --> i do not give a damn
idhac --> i do not have a clue
ilyaaf --> i love you as a friend
ilyam --> i love you as a mate
ilylas --> i love you like 

In [ ]:
# TODO: ES search function on searching abbreviation of phrases
""" 
"""
def search_es_abbreviation(list_words):
    
    return abbreviation->str

In [ ]:
"""search in abbreviation database based on shingles,
return slang and the corresponding shingle."""

def abbreviation_search(shingle_lst):
    lst_search_result = []
    for i in range(len(shingle_lst)):
        list_temp = shingle_lst[i]
        for j in range(len(list_temp)):
            word_temp = list_temp[j]
            search_result = search_es_abbreviation(word_temp)
            # TODO: make it a fuzzy search
            if search_result: # when search result non empty
                # give list of paired (word, search result)
                lst_search_result.append((word_temp, search_result))
    return lst_search_result

In [ ]:
""" use Urban dictionary to perform fuzzy search.
TODO
NOTE: another possible way is to bring the word to the phrase,
namely extract keyword from urban dictionary&match it with the phrase."""
def UD_slang_search(phrase_list):
    return slang_list
    # formulation of slang_list: [(slang, word/phrase)]

In [ ]:
"""search in UD database based on phrases,
return slang and the corresponding phrase."""

def slang_search(phrase_lst):
    lst_search_result = []
    for i in range(len(phrase_lst)):
        list_temp = phrase_lst[i]
        for j in range(len(list_temp)):
            word_temp = list_temp[j]
            search_result = UD_slang_search(word_temp)
            # TODO: make it a fuzzy search
            if search_result: # when search result non empty
                # give list of paired (word, search result)
                lst_search_result.append((word_temp, search_result))
    return lst_search_result

In [4]:
  
index_name = "emo31"
request = '''{"query": {"match_all": {}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'emo31', '_type': 'emojis', '_id': 'nr2sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'index': {'_index': 'emo31'}}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'n72sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'id': 0, 'emoji': '😀', 'meaning': ['grinning_face', 'face', 'smile', 'happy', 'joy', ':D', 'grin']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'oL2sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'index': {'_index': 'emo31'}}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'ob2sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'id': 1, 'emoji': '😃', 'meaning': ['grinning_face_with_big_eyes', 'face', 'happy', 'joy', 'haha', ':D', ':)', 'smile', 'funny']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'or2sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'index': {'_index': 'emo31'}}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'o72sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'id': 2, 'emoji': '😄', 'meaning': ['grinning_face_with_smiling_eyes', 'face', 'happy', '

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
#ES search query for a specific word (testcase)
request = '''{"query": {"match": { "meaning":"kiss"}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'emo31', '_type': 'emojis', '_id': 'J72sFYUBDqVhmJIZoIW7', '_score': 6.634506, '_source': {'id': 196, 'emoji': '👄', 'meaning': ['mouth', 'mouth', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'R4anrIUBOSZGbDlKy07z', '_score': 6.634506, '_source': {'id': 196, 'emoji': '👄', 'meaning': ['mouth', 'mouth', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'db2sFYUBDqVhmJIZoIS7', '_score': 6.0449953, '_source': {'id': 107, 'emoji': '😽', 'meaning': ['kissing_cat', 'animal', 'cats', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'lYanrIUBOSZGbDlKy03z', '_score': 6.0449953, '_source': {'id': 107, 'emoji': '😽', 'meaning': ['kissing_cat', 'animal', 'cats', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'x72sFYUBDqVhmJIZoIO7', '_score': 5.132836, '_source': {'id': 20, 'emoji': '😙', 'meaning': ['kissing_face_with_smiling_eyes', 'face', 'affection', 'valentines', 'infatuation', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': '54anrIUBOSZGbDl

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
[d['_source']['meaning'] for d in results]

[['mouth', 'mouth', 'kiss'],
 ['mouth', 'mouth', 'kiss'],
 ['kissing_cat', 'animal', 'cats', 'kiss'],
 ['kissing_cat', 'animal', 'cats', 'kiss'],
 ['kissing_face_with_smiling_eyes',
  'face',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['kissing_face_with_smiling_eyes',
  'face',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['kiss', 'pair', 'valentines', 'love', 'like', 'dating', 'marriage'],
 ['kiss', 'pair', 'valentines', 'love', 'like', 'dating', 'marriage'],
 ['face_blowing_a_kiss',
  'face',
  'love',
  'like',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['kissing_face',
  'love',
  'like',
  'face',
  '3',
  'valentines',
  'infatuation',
  'kiss'],
 ['kissing_face_with_closed_eyes',
  'face',
  'love',
  'like',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['face_blowing_a_kiss',
  'face',
  'love',
  'like',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['kissing_face',
  'love',
  'like',
  'face',
  '

In [ ]:
# TODO: ES search in emoji database and return 
# list of corresponding emojis
def emoji_search_es(word):
    return emoji_list->list
    # return a list of emoji searched
    # can be UTF-8 encoded or numbered according to order in dataset

In [34]:
"""remove stop words,
perform a word association based on twitter corpus.
perform ES query based on associated words.
return: corresponding word and associated emoji as list of tuple."""


def emoji_search(input_str):
    emoji_search_results = []
    en_stops = nltk.corpus.stopwords.words("english")
    word_list = [x.strip() for x in input_str.split(" ")]
    model = Word2Vec.load('model.bin')
    for word in word_list:
        emoji_search_temp = []
        if word not in en_stops:
            # perform association
            # perform emoji search
            list_of_emoji_original = emoji_search_es([word])
            emoji_search_temp = emoji_search_temp + list_of_emoji_original
            if not len(list_of_emoji_original):
                # if the origial word cannot be found
                association_list = model.wv.most_similar(word)
                for word_associated in association_list:
                # store in list_of_emoji
                    list_of_emoji = emoji_search_es(word_associated)
                    emoji_search_temp = emoji_search_temp + list_of_emoji
        emoji_search_results.append((word, emoji_search_temp))
    return emoji_search_results
    # TODO: can also based on api to reversedictionary.com
    # which gives associated words to noun phrases: 
    # 1. extract noun phrases with nlp().noun_chunks,2.search
    # the phrase up in reversedictionary.com, and
    # 3.perform query with ES in emoji dataset with obtained 
    # set of words.

In [ ]:
def emoji_search_alternative(word_lst):
    emoji_search_results = []
    for word in word_lst:
        emoji_search_temp = []
            # perform association
        association_list = model.wv.most_similar(word)
            # perform emoji search
        for word_associated in association_list:
            # store in list_of_emoji
            list_of_emoji = emoji_search_es(word_associated)
        # TODO: return word_associated with associated emoji
            emoji_search_temp = emoji_search_temp + list_of_emoji
        emoji_search_results.append((word, emoji_search_temp))
    return emoji_search_results

In [ ]:
"""The general function that would 
translate input to sentence with slang&emoji.
return&print translated sentence.
To be continued.
"""
def translation_style(input_str):
    # for each of the words perform emoji search first
    # if we want the precise results of emoji association,
    # it would be best to not perform association based on trivial words
    # -> STOP WORD REMOVAL
    emoji_list = emoji_search(input_str)
    # completely word based so we might just 
    # append the emoji after words.
    # another way is to only perform association 
    # when you encounter a certain grammartical structure
    # for eg noun phrase&verb phrase. 
    str_doc = nlp(input_str)
    word_list = []
    for word in str_doc:
        if word.pos_ == "VERB" or word.pos_ == "NOUN" or word.pos_ == "ADJ":
            word_list.append(word)
    emoji_list_alternative = emoji_search_alternative(word_list)
    # need to note the order of the words since same word
    # can appear multiple times as multiple POS
    phrase_list = phrase_extraction(str_doc)
    
    shingle_lst = multiple_shingle(input_str)
    # the problem here is that after translating input
    # to slangs the word formulation would differ.
    # ->should make sure which part translates to which
    abbrev_list = abbreviation_search(shingle_lst)
    # search for further slangs with the original sentence
    # and if the slang results overlap with the abbreviations
    # for eg got 2 slangs for same word
    # then use any one with equal probability
    slang_list = slang_search(phrase_list)
    # TODO: get the results together:
    # append the emojis->
    # see if the abbrev list clashes with the UD one->
    # decide the final list->
    # substitute words with slangs
    return slang_sentence


    
    
    